In [7]:
# Imports

import os
import pandas as pd
import numpy as np
import copy

import torch
from torch import nn, optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, validation_curve, GridSearchCV, learning_curve
from sklearn.metrics import mean_squared_error,r2_score
from skorch import NeuralNetRegressor
import seaborn as sns

In [8]:
# Load data

# load
df = pd.read_csv("data/satellite1_1000.csv")
df.replace([np.inf, -np.inf], np.nan, inplace=True)
df=df.dropna()

# get input variables
X = df[df.columns[0:-4]]

# get output variable
density = df['log_density'] # log based, use 'density' if want density value instead of logged.
perturbation = df['perturbation']
norm_perturbation = df['norm_perturbation']

# transform
xscaler = preprocessing.MinMaxScaler()
names = X.columns
d = xscaler.fit_transform(X)
X = pd.DataFrame(d, columns=names)

yscaler = preprocessing.MinMaxScaler()
d = yscaler.fit_transform(perturbation.values.reshape(-1, 1))
y = pd.DataFrame(d, columns=['norm_perturbation'])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.4, random_state = 628)
# Convert to 2D PyTorch tensors
X_train = torch.tensor(X_train.values, dtype=torch.float32)
y_train = torch.tensor(y_train.values, dtype=torch.float32).reshape(-1, 1)
X_test = torch.tensor(X_test.values, dtype=torch.float32)
y_test = torch.tensor(y_test.values, dtype=torch.float32).reshape(-1, 1)

In [ ]:
# Optional, skip for multiple run

dataset = df.copy()
_ = sns.pairplot(
    dataset[['mlat', 'cos', 'sin', 'rho', 'ae_index', 'sym_h','density']],
    kind='reg', diag_kind='kde', plot_kws={'scatter_kws': {'alpha': 0.1}})

In [9]:
# Check Device

device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

# define neural network model
model = nn.Sequential(
    nn.Linear(126, 10),
    nn.ReLU(),
    nn.Linear(10, 20),
    nn.ReLU(),
    nn.Linear(20, 1)
)

# create skorch wrapper for a regressor.
netRegressor = NeuralNetRegressor(
    module=model,
    criterion=nn.MSELoss,
    optimizer=optim.Adam,
    max_epochs=32,
    batch_size=128,
    device=device
)

# Use GridSearchCV to perform hyperparameter tuning
param_grid = {
    'batch_size': [32, 64, 128],
    'criterion': [nn.MSELoss]
}

# uncomment below to parameter tuning

# grid_search = GridSearchCV(estimator=net, param_grid=param_grid, cv=5, scoring='r2', n_jobs=-2)
# print("Best parameters found:", grid_search.best_params_)
# print("Best score:", grid_search.best_score_)

Using cuda device


In [10]:
# Save prediction result
netRegressor.fit(X_train, y_train)
# print(f'model score on testing data: {netRegressor.score(X_test, y_test)}')
y_train_out = netRegressor.predict(X_train)
y_out = netRegressor.predict(X_test)

  epoch    train_loss    valid_loss     dur
-------  ------------  ------------  ------
      1        0.0068        0.0048  0.1935
      2        0.0039        0.0295  0.1819
      3        0.0044        0.0111  0.1856
      4        0.0033        0.0050  0.1857
      5        0.0026        0.0053  0.1830
      6        0.0027        0.0048  0.1840
      7        0.0027        0.0035  0.1820
      8        0.0025        0.0033  0.1859
      9        0.0025        0.0032  0.1863
     10        0.0025        0.0030  0.1819
     11        0.0025        0.0029  0.1801
     12        0.0025        0.0028  0.1839
     13        0.0025        0.0029  0.1819
     14        0.0025        0.0029  0.1860
     15        0.0024        0.0030  0.1883
     16        0.0025        0.0028  0.1880
     17        0.0024        0.0029  0.1813
     18        0.0025        0.0028  0.1848
     19        0.0025        0.0030  0.1838
     20        0.0025        0.0028  0.1677
     21        0.0025        0.0

In [11]:
print(r2_score(y_test, y_out))

0.543667772108455


In [12]:
pd.DataFrame(X_test).describe()

,0,1,2,3,4,5,6,7,8,9,...,116,117,118,119,120,121,122,123,124,125
count,7264.000000,7264.000000,7264.000000,7264.000000,7264.000000,7264.000000,7264.000000,7264.000000,7264.000000,7264.000000,...,7264.000000,7264.000000,7264.000000,7264.000000,7264.000000,7264.000000,7264.000000,7264.000000,7264.000000,7264.000000
mean,0.502356,0.478638,0.493995,0.654922,0.081415,0.073956,0.062908,0.078699,0.070942,0.062745,...,0.754171,0.744362,0.724646,0.679459,0.676933,0.661927,0.668634,0.650735,0.658800,0.675288
std,0.250289,0.351906,0.354548,0.214619,0.097452,0.087861,0.075460,0.093677,0.084300,0.074515,...,0.068921,0.068356,0.069225,0.065917,0.067852,0.066163,0.066713,0.065855,0.070000,0.073608
min,0.000682,0.000000,0.000000,0.028552,0.000000,0.000000,0.000000,0.000000,0.000517,0.000000,...,0.089782,0.103431,0.088197,0.088791,0.071977,0.090283,0.126887,0.113453,0.102369,0.073167
25%,0.315075,0.128205,0.138674,0.497013,0.019524,0.018171,0.015057,0.019262,0.017586,0.015124,...,0.724280,0.714728,0.694609,0.650822,0.647482,0.633088,0.639509,0.622047,0.628422,0.643441
50%,0.500302,0.467059,0.490769,0.712154,0.041155,0.037886,0.031953,0.040481,0.036352,0.031792,...,0.765432,0.755481,0.735861,0.690175,0.688011,0.672656,0.679444,0.661417,0.670173,0.687213
75%,0.690185,0.824615,0.848144,0.837949,0.105792,0.094335,0.081059,0.101439,0.091315,0.081885,...,0.796656,0.786391,0.766383,0.718127,0.716381,0.700507,0.708753,0.690178,0.702581,0.722230
max,0.998609,1.000000,1.000000,1.000000,0.809727,0.774215,0.837211,1.000000,0.953857,0.723567,...,0.971193,0.964601,0.955794,0.901707,0.902812,0.877168,0.877863,0.862205,0.885964,0.931177


In [ ]:
pd.DataFrame(y_out).describe()

In [ ]:
pd.DataFrame(y_test).describe()

In [15]:
X_tmp = xscaler.inverse_transform(X_test)
pd.DataFrame(X_tmp).describe()

,0,1,2,3,4,5,6,7,8,9,...,116,117,118,119,120,121,122,123,124,125
count,7264.000000,7264.000000,7264.000000,7264.000000,7264.000000,7264.000000,7264.000000,7264.000000,7264.000000,7264.000000,...,7264.000000,7264.000000,7264.000000,7264.000000,7264.000000,7264.000000,7264.000000,7264.000000,7264.000000,7264.000000
mean,0.127391,-0.042724,-0.012010,4.766711,166.669290,166.479489,166.336838,166.389156,166.145738,165.800317,...,-10.736403,-10.728371,-10.718609,-10.722862,-10.733360,-10.711510,-10.706922,-10.713285,-10.723882,-10.724249
std,9.940806,0.703812,0.709095,1.089847,194.486599,194.215623,194.498640,194.485021,193.864734,192.150507,...,16.747761,16.773171,16.780895,16.750056,16.741882,16.721435,16.705625,16.727162,16.766298,16.816492
min,-19.797705,-1.000000,-1.000000,1.585968,4.189233,3.000000,4.189233,3.000000,4.189233,4.000000,...,-172.182900,-168.000001,-165.000000,-160.817101,-160.000001,-155.182900,-146.365798,-147.182900,-144.000000,-148.284267
25%,-7.310890,-0.743590,-0.722652,3.964841,43.154113,43.166678,42.999999,42.991000,43.442576,43.000000,...,-18.000004,-18.000006,-17.999999,-17.999997,-18.000000,-17.999993,-17.999994,-18.000000,-17.999999,-18.000005
50%,0.045799,-0.065882,-0.018462,5.057341,86.322092,86.746118,86.547402,87.043969,86.598202,85.980916,...,-7.999995,-7.999997,-7.999996,-7.999999,-8.000000,-8.000004,-7.999994,-8.000004,-7.999995,-7.999995
75%,7.587450,0.649229,0.696287,5.696137,215.319795,211.527460,213.119159,213.600070,212.997472,215.157581,...,-0.412498,-0.415248,-0.601088,-0.896972,-1.000003,-0.961302,-0.660534,-0.694676,-0.237551,-0.000001
max,19.837205,1.000000,1.000000,6.519042,1620.171815,1714.400166,2162.115564,2079.131833,2196.577498,1869.860193,...,42.000004,43.313868,45.313868,45.752407,45.000004,43.686139,41.686133,43.000002,43.686132,47.735856


In [20]:
dnyp = yscaler.inverse_transform(y_out).flatten()
dnyo = yscaler.inverse_transform(y_test).flatten()

In [22]:
pd.DataFrame(dnyo).describe()

,0
count,7264.000000
mean,23.603221
std,42.452590
min,0.032950
25%,0.753141
50%,5.166594
75%,28.731738
max,554.070820


In [21]:
pd.DataFrame(dnyp).describe()

,0
count,7264.000000
mean,28.505459
std,36.372150
min,3.376600
25%,3.376600
50%,5.776438
75%,45.473007
max,152.476578


In [23]:
# output the results

pd.DataFrame(X_tmp).to_csv('inputs.csv', index=False)
pd.DataFrame(dnyo).to_csv('original_output.csv', index=False)
pd.DataFrame(dnyp).to_csv('predicted_output.csv', index=False)


In [ ]:
pd.DataFrame(dny).describe()

In [ ]:
plt.figure(figsize=(10,10))
plt.scatter(y_test.numpy(), y_out, c='crimson')
# plt.yscale('log')
# plt.xscale('log')

# p1 = max(max(dny), max(y_test.numpy()))
# p2 = min(min(dny), min(y_test.numpy()))
# plt.plot([p1, p2], [p1, p2], 'b-')
plt.xlabel('True Values', fontsize=15)
plt.ylabel('Predictions', fontsize=15)
plt.axis('equal')
plt.show()

In [ ]:
# full data run

full_df = pd.read_csv("den_data.csv")
full_df.replace([np.inf, -np.inf], np.nan, inplace=True)
full_df=full_df.dropna()

X = full_df[full_df.columns[0:-1]]  #  X will hold all features
#y = df['theta5']/np.pi # y will hold target/labels
y = np.log10(full_df['density'])
scaler = preprocessing.MinMaxScaler()
names = X.columns
d = scaler.fit_transform(X)
X = pd.DataFrame(d, columns=names)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0, random_state = 628)
# Convert to 2D PyTorch tensors
X_train = torch.tensor(X_train.values, dtype=torch.float32)
y_train = torch.tensor(y_train.values, dtype=torch.float32).reshape(-1, 1)
X_test = torch.tensor(X_test.values, dtype=torch.float32)
y_test = torch.tensor(y_test.values, dtype=torch.float32).reshape(-1, 1)

X_train.describe()